In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
import time

start = time.time()

train_image_path = '../input/resized-plant2021/img_sz_256/' #uses smaller version of dataset for efficiency
test_image_path = '../input/plant-pathology-2021-fgvc8/test_images/'
train_df_path = '../input/plant-pathology-2021-fgvc8/train.csv'
test_df_path = '../input/plant-pathology-2021-fgvc8/sample_submission.csv'

train_df = pd.read_csv(train_df_path)

In [ ]:
blue_minimums = []
blue_maximums = []
blue_means = []

green_minimums = []
green_maximums = []
green_means = []

red_minimums = []
red_maximums = []
red_means = []


for image_path in train_df['image'].tolist():
    img = cv2.imread(train_image_path + image_path) #cv2 reads image into numpy array
    #openCV uses BGR image formatting, so
    blue_channel = img[:,:,0]
    green_channel = img[:,:,1]
    red_channel = img[:,:,2]
    

    
    #extract features
    blue_minimums.append(np.min(blue_channel))
    blue_maximums.append(np.max(blue_channel).astype(np.int16))
    blue_means.append(np.mean(blue_channel))
    
    green_minimums.append(np.min(green_channel))
    green_maximums.append(np.max(green_channel).astype(np.int16))
    green_means.append(np.mean(green_channel))

    red_minimums.append(np.min(red_channel))
    red_maximums.append(np.max(red_channel).astype(np.int16))
    red_means.append(np.mean(red_channel))

In [ ]:
colour_features = pd.DataFrame(
    {     
    'blue_minimums' : blue_minimums,
    'blue_maximums' : blue_maximums,
    'blue_means' : blue_means,
    
    'green_minimums' : green_minimums,
    'green_maximums' : green_maximums,
    'green_means' : green_means,

    'red_minimums' : red_minimums,
    'red_maximums' : red_maximums,
    'red_means' : red_means
    })

y_train = train_df['labels'].astype('category')

linreg = LogisticRegression(solver='liblinear')
linreg.fit(colour_features, y_train)

In [ ]:
print(colour_features)
weight = linreg.coef_ 
print(weight)

In [ ]:
test_df = pd.read_csv(test_df_path)

blue_minimums = []
blue_maximums = []
blue_means = []

green_minimums = []
green_maximums = []
green_means = []

red_minimums = []
red_maximums = []
red_means = []

for image_path in test_df['image'].tolist():
    img = cv2.imread(test_image_path + image_path) #cv2 reads image into numpy array
    #openCV uses BGR image formatting, so
    blue_channel = img[:,:,0]
    green_channel = img[:,:,1]
    red_channel = img[:,:,2]
    
    #extract features
    blue_minimums.append(np.min(blue_channel))
    blue_maximums.append(np.max(blue_channel).astype(np.int16))
    blue_means.append(np.mean(blue_channel))
    
    green_minimums.append(np.min(green_channel))
    green_maximums.append(np.max(green_channel).astype(np.int16))
    green_means.append(np.mean(green_channel))

    red_minimums.append(np.min(red_channel))
    red_maximums.append(np.max(red_channel).astype(np.int16))
    red_means.append(np.mean(red_channel))
    
colour_features = pd.DataFrame(
        {     
        'blue_minimums' : blue_minimums,
        'blue_maximums' : blue_maximums,
        'blue_means' : blue_means,

        'green_minimums' : green_minimums,
        'green_maximums' : green_maximums,
        'green_means' : green_means,

        'red_minimums' : red_minimums,
        'red_maximums' : red_maximums,
        'red_means' : red_means
        })    

preds = linreg.predict(colour_features)
test_df['labels'] = preds
    
test_df.to_csv('submission.csv', index=False)

end = time.time()
print(end - start)